In [ ]:
! pip install mlx_lm

In [ ]:
! pip install langchain

In [ ]:
! pip install sentence-transformers

In [ ]:
! pip install torch

In [ ]:
pip install pdfplumber

Note: you may need to restart the kernel to use updated packages.


In [ ]:
! pip install datasets


In [ ]:
! pip install streamlit

In [ ]:
! pip install scikit-learn pandas huggingface-hub

In [ ]:
! pip install "transformers[torch]" accelerate
! pip install 'accelerate>=0.26.0'

In [ ]:
from mlx_lm import load, generate
import pdfplumber
from sentence_transformers import SentenceTransformer
import numpy as np
from langchain.text_splitter import RecursiveCharacterTextSplitter
import streamlit as st
from mlx_lm.models.cache import make_prompt_cache
from mlx_lm.sample_utils import make_sampler
from mlx_lm.generate import stream_generate
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer
import accelerate
from transformers import TrainingArguments, Trainer
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from transformers import pipeline


In [ ]:
pdf_path = "Data_Activisim_Piechart_Activity.pdf"

text_chunks = []
with pdfplumber.open(pdf_path) as pdf:
    for page in pdf.pages:
        text = page.extract_text()
        if text:
            text_chunks.append(text)

###########################################
# STEP 2: Split text into manageable chunks
###########################################

splitter = RecursiveCharacterTextSplitter(
    chunk_size=800,
    chunk_overlap=50
)

docs = splitter.create_documents(text_chunks)

###########################################
# STEP 3: Embed chunks for retrieval
###########################################

embedder = SentenceTransformer("all-MiniLM-L6-v2")
doc_texts = [doc.page_content for doc in docs]
embeddings = embedder.encode(doc_texts)

###########################################
# STEP 4: Define retrieval function
###########################################

def retrieve_context(question, docs, embeddings, embedder, top_k=1):
    q_emb = embedder.encode([question])[0]
    similarities = np.dot(embeddings, q_emb)
    top_k_idx = similarities.argsort()[-top_k:][::-1]
    return [docs[i].page_content for i in top_k_idx]


In [ ]:
model, tokenizer = load ("google/gemma-3-1b-it")

if "prompt_cache" not in st.session_state:
    st.session_state.prompt_cache = make_prompt_cache(model, max_kv_size=2048)

# Streamlit app setup
st.title("AVELA Data Activism Assistant")



#question = st.text_input("Ask your question about data activism or code hints:")

question = f""" how can i make a data visualization """

classifier_model_path = "./data_activism_classifier"
clf_tokenizer = AutoTokenizer.from_pretrained(classifier_model_path)
clf_model = AutoModelForSequenceClassification.from_pretrained(classifier_model_path)

clf = pipeline(
    "text-classification",
    model="./data_activism_classifier",
    tokenizer=clf_tokenizer,
    return_all_scores=False
)

def classify(text):
    out = clf(text)[0]
    label_id = int(out["label"].split("_")[-1]) if out["label"].startswith("LABEL_") else out["label"]
    return "on-topic" if label_id in (1, "1") else "off topic"

if(question):
    if(classify(question)=="on-topic"):
        print("on-topic")
        if question:
            context_chunks = retrieve_context(question, docs, embeddings, embedder)
            context_text = "\n".join(context_chunks)

            messages = [
            {
                "role": "user",
                "content": f"""You are an expert data activism and programming tutor for high school students.

                Your role is to provide step-by-step hints to guide students in completing coding and data visualization tasks. You do not write full solutions or complete the code for them. Instead, you offer clear, concise, and detailed hints for only the next step they should take, focusing on replacing any placeholder question marks (?) in their code.

                Always ensure:
                - Your responses are limited to topics related to data activism, coding, and data visualization.
                - You avoid answering any questions unrelated to these topics.
                - Your hints are detailed enough for the student to take confident action but never include the full solution.
                - If a student asks about definitions or programming concepts related to data activism, provide direct and clear explanations.
                - Encourage students to ask more specific questions about data activism if they ask general or unrelated questions.
                Respond using no more than 5 clear and precise sentences per hint. Do not include any code snippets, code blocks, or print statements in your response unless explicitly asked for code.

                Context:
                {context_text}

                Task:
                Question: {question}"""
                    }
                ]

            prompt = tokenizer.apply_chat_template(messages, add_generation_prompt=True)

            ###########################################
            # STEP 7: Generate response using MLX
            ###########################################


            response_text = ""
            for response in stream_generate(
                model,
                tokenizer,
                prompt,
                max_tokens=1024,
                #sampler=make_sampler(temp=0.0, top_p=1.0),
                prompt_cache=st.session_state.prompt_cache,
            ):
                response_text += response.text

            #st.write(response_text)
            print(response_text)
    else:
            print("off-topic")
            messages = [
            {
                "role": "user",
                "content": f"""You are an expert data‑activism and programming tutor for high‑school students.
                    You only discuss coding, data visualization and other topics directly related to data activism.
                    If the student asks anything off topic, respond in exactly two sentences:
                    1) remind them that we’re focusing on data‑activism tasks;
                    2) invite them to ask a new question about data activism.
                    Do not give any definitions.
                Task:
                Question: {"student just asked an off topic question, guide them back to data activism?"}"""
                    }
                ]

            prompt = tokenizer.apply_chat_template(messages, add_generation_prompt=True)

            ###########################################
            # STEP 7: Generate response using MLX
            ###########################################


            response_text = ""
            for response in stream_generate(
                model,
                tokenizer,
                prompt,
                max_tokens=1024,
                #sampler=make_sampler(temp=0.0, top_p=1.0),
                prompt_cache=st.session_state.prompt_cache,
            ):
                response_text += response.text
            print(response_text)
            #st.write(response_text)



Fetching 8 files:   0%|          | 0/8 [00:00<?, ?it/s]

2025-07-22 12:11:02.313 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-22 12:11:02.319 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-22 12:11:02.319 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-22 12:11:02.319 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
Device set to use mps:0


on-topic


2025-07-22 12:11:06.877 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


Okay, let’s start with the next step.  You’ll need to create a bar graph using a library like Matplotlib or Seaborn.  First, make sure you’ve imported the necessary libraries – `matplotlib` or `seaborn` – into your notebook.  Then, load your data into a Pandas DataFrame.
